<a href="https://colab.research.google.com/github/AlexH20/NLP_Sentiment_Methods/blob/main/GetdataAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wrds
!pip install pandas_market_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 14.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 7.3 MB/s 
     |████████████████████████████████| 152 kB 36.0 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.3-py3-none-any.whl size=182636 sha256=0a6f4034675a3633dfffcc17a0ea93c6720d53bb0829870d8cd1b1b0981f8b50
  Stored in directory: /root/.cache/pip/wheels/b8/a3/19/b4611514d34ffd61d13aef10fefc2dcaf3754145121ceba647
Successfully built exchange-calendars


In [ ]:
import wrds
import csv
from datetime import datetime, timedelta, date
import pandas as pd
from google.colab import drive 
import holidays
import pandas_market_calendars as mcal

drive.mount("/content/gdrive")

db = wrds.Connection(wrds_username='holzeral')

holidaysUS = holidays.US()
nyse = mcal.get_calendar('NYSE')
stock_holidays = nyse.holidays()

stock_holidays = list(pd.to_datetime(stock_holidays.holidays))
stock_holidays = [x.date() for x in stock_holidays]

def check_trading_dayandhour(day):
    trading_day = day
    while trading_day.weekday() in holidays.WEEKEND or trading_day in stock_holidays:
        trading_day += timedelta(1)
    return trading_day


data = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woS.csv")
data["Date"] = pd.to_datetime(data["Date"])
data["Year"] = [x.year for x in data["Date"]]
data["Month"] = [x.month for x in data["Date"]]

data = data[~data["Year"].isin([2009,2010,2011,2012,2013,2014,2015,2016])]

print(data.head())

data_fill = []

i = 0
i_pos = 0
i_neg = 0

for index, row in data.iterrows():

      print(i)
      i += 1

      tick = row["Ticker"]
      text = row["Text"]
      date_adj = check_trading_dayandhour(row["Date"])

      day0 = date_adj - timedelta(0)

      if date_adj.weekday() != 4:
          dayplus1 = date_adj + timedelta(1)
      else:
          dayplus1 = date_adj + timedelta(3)

      dayplus1 = check_trading_dayandhour(dayplus1)
      dayplus1 = dayplus1.strftime("%m/%d/%Y")
      day0 = day0.strftime("%m/%d/%Y")


      permno = db.raw_sql("""select permno, date
                                                from crsp.dse
                                                where TICKER = '{}'
                                                and date  <= '{}'""".format(tick, day0))
      

      #Get latest permno
      try:
        permno["date"] = pd.to_datetime(permno["date"], format = "%Y/%m/%d")
      except:
        continue
      
      permno = permno.sort_values(by = "date")
      permno = permno["permno"].iloc[-1]

      # Fetch prices and returns for given buy and hold period
      exc_ret = db.raw_sql("""select date, prc, ret
                                                          from crsp.dsf
                                                          where permno = {}
                                                          and date = '{}'""".format(permno, day0))

      vw_ret = db.raw_sql("""select date, vwretd
                                                          from crsp.dsi
                                                          where date = '{}'""".format(day0))
      

      ar = exc_ret["ret"][0] - vw_ret["vwretd"][0]

      data_fill.append([row["Date"], tick, row["Nasdaq"], row["Size"], row["BTM"], row["pref_alpha"], row["Text"], row["CAR"], ar])

df = pd.DataFrame(data_fill, columns = ["Date", "Ticker", "Nasdaq", "Size", "BTM", "pref_alpha", "Text", "CAR", "AR"])

with open("gdrive/My Drive/Thesis/processed data/CAR_regression/data_whole_wScARn.csv", "w") as csv_file:
            
            writer = csv.writer(csv_file)
            writer.writerow(
                ["Date", "Ticker", "Nasdaq", "Size", "BTM", "pref_alpha", "Text", "CAR", "AR"])
            for index, row in df.iterrows():
                writer.writerow([row["Date"], row["Ticker"], row["Nasdaq"], row["Size"], row["BTM"], row["pref_alpha"], row["Text"], row["CAR"], row["AR"]])


      


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading library list...
Done
          Date Ticker  Nasdaq  Turnover          Size       BTM  pref_alpha  \
522 2017-01-02   AAPL       1  1.192661  6.106953e+08  0.219499    0.000949   
523 2017-01-03   AAPL       1  1.192661  6.106953e+08  0.219499    0.000949   
524 2017-01-04   AAPL       1  1.184873  6.106953e+08  0.219499    0.000943   
525 2017-01-05   AAPL       1  1.168894  6.100118e+08  0.219745    0.001215   
526 2017-01-06   AAPL       1  1.153252  6.131139e+08  0.218633    0.001369   

          CAR                                               Text  word_count  \
522 -0.015182                                                              0   
523 -0.015182  applenasdaq aapl finished 00NUMBER00 strong ga...         720   
524 -0.003706  anecdotal evidence is often of limited use whe...         892   
525  0.014927  image source the motley fool. 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
